<a href="https://colab.research.google.com/github/clento173/OpenSourceDataAnalysis_3-2/blob/main/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import requests
import pandas as pd
import json

url = "http://openapi.seoul.go.kr:8088/4d54555a6965756e3130326d6c4c5867/json/energyUseDataSummaryInfo/1/100/"
responses = []
for year in range(2015,2016):
    for month in range(1,13):
        url_month = url + str(year) + "/" + "{:02d}".format(month)
        response = requests.get(url_month).json()
        responses.append(select_col)

#data = responses.json()

#data = json.loads(responses)

#key = list(responses.json().keys())[0]
#data = responses.json().get(key).get('row')

print(responses)
 # key = list(response.json().keys())[0]
 # data = response.json().get(key).get('row')
 # df = pd.DataFrame(data)


#url = " http://openapi.seoul.go.kr:8088/4d54555a6965756e3130326d6c4c5867/json/energyUseDataSummaryInfo/1/100/"
#api_key = "4d54555a6965756e3130326d6c4c5867"

#response = requests.get(url)

#if response.status_code == 200:
#  print("api 호출 성공")
#  print(response.json())
#else:
#  print(f"API 호출 실패: {response.status_code}")

['{ EUS : 193784708, GUS : 59133720, WUS : 12819757.886, HUS : 22740838.937}', '{ EUS : 189974230, GUS : 56487358, WUS : 12656888.218, HUS : 18793320.213}', '{ EUS : 145577747, GUS : 7601585, WUS : 2570265.8, HUS : 48}', '{ EUS : 134199179, GUS : 5544226, WUS : 3003915.8, HUS : 9}', '{ EUS : 177289769, GUS : 22060892, WUS : 13086734.214, HUS : 1320895.801}', '{ EUS : 123484926, GUS : 3244802, WUS : 3276567.2, HUS : 0}', '{ EUS : 556996768, GUS : 12877575, WUS : 4433516.8, HUS : 0}', '{ EUS : 242000479, GUS : 7619018, WUS : 15164553.742, HUS : 460288.965}', '{ EUS : 117000086, GUS : 3130770, WUS : 2286005, HUS : 0}', '{ EUS : 186473958, GUS : 8924018, WUS : 14711828.766, HUS : 3167520.127}', '{ EUS : 147686501, GUS : 2443388, WUS : 1424682, HUS : 0}', '{ EUS : 203388142, GUS : 40095231, WUS : 13904162.958, HUS : 21102616.717}']
